# State Management Demo: Insurance Claim Processing

## Overview

This demo applies the same state management pattern to **insurance claim processing** 

This shows that state management is a **universal pattern** for multi-step workflows.

## Claim Processing States

1. **INTAKE** - Collect and validate claim information
2. **ASSESSMENT** - AI reviews claim and estimates validity
3. **VERIFICATION** - Check against policy and fraud rules
4. **DECISION** - Approve, deny, or request more info

## Setup

In [1]:
import os
from dataclasses import dataclass, field
from enum import Enum
from typing import List
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client with Vocareum endpoint
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Ready!")

✅ Ready!


## Define the State Machine

In [2]:
class ClaimState(Enum):
    INTAKE = "intake"
    ASSESSMENT = "assessment"
    VERIFICATION = "verification"
    DECISION = "decision"
    COMPLETED = "completed"

# Sample insurance claims
CLAIMS = [
    {
        "claim_id": "CLM-001",
        "customer": "John Smith",
        "claim_type": "Auto Accident",
        "amount": 5000,
        "description": "Minor fender bender, passenger side damage. Two-year customer, clean driving record.",
        "policy_active": True
    },
    {
        "claim_id": "CLM-002",
        "customer": "Jane Doe",
        "claim_type": "Water Damage",
        "amount": 35000,
        "description": "Pipe burst in basement. New customer (1 month), first claim ever.",
        "policy_active": True
    },
]

print("✅ States and sample claims defined!")

✅ States and sample claims defined!


## Define Context (State Preservation)

In [3]:
@dataclass
class ClaimContext:
    """Claim state maintained across processing steps"""
    claim_id: str
    customer: str
    claim_type: str
    amount: float
    description: str
    policy_active: bool
    
    # Results from each step
    intake_valid: bool = False
    ai_assessment: str = ""  # LLM response
    fraud_score: float = 0.0
    final_decision: str = ""
    
    # Workflow tracking
    current_state: ClaimState = ClaimState.INTAKE
    state_history: List[str] = field(default_factory=list)
    
    def transition(self, new_state: ClaimState):
        """Record state transition"""
        self.state_history.append(f"{self.current_state.value} → {new_state.value}")
        self.current_state = new_state

print("✅ Context defined!")

✅ Context defined!


## Build the Workflow

In [4]:
class ClaimProcessor:
    """Stateful insurance claim processing"""
    
    def __init__(self, claim: dict):
        self.context = ClaimContext(
            claim_id=claim["claim_id"],
            customer=claim["customer"],
            claim_type=claim["claim_type"],
            amount=claim["amount"],
            description=claim["description"],
            policy_active=claim["policy_active"]
        )
    
    def intake(self):
        """Step 1: Intake and validation"""
        print(f"\n1️⃣ INTAKE: Validating claim {self.context.claim_id}...")
        
        self.context.intake_valid = (
            self.context.policy_active and
            self.context.amount > 0 and
            len(self.context.description) > 0
        )
        
        if self.context.intake_valid:
            print(f"   ✅ Valid claim intake")
            print(f"      • Claim: {self.context.claim_type}")
            print(f"      • Amount: ${self.context.amount:,.0f}")
        else:
            print(f"   ❌ Invalid claim")
            return False
        
        self.context.transition(ClaimState.ASSESSMENT)
        return True
    
    def assess(self):
        """Step 2: AI assessment using context from Step 1"""
        print(f"\n2️⃣ ASSESSMENT: AI analyzing claim...")
        
        # Use verified data from Step 1
        prompt = f"""Assess this insurance claim. Respond in 2 sentences with fraud risk assessment.

Claim ID: {self.context.claim_id}
Type: {self.context.claim_type}
Amount: ${self.context.amount:,.0f}
Details: {self.context.description}

Provide: Risk assessment (LOW/MEDIUM/HIGH) and key red flags (if any)."""
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            max_tokens=100,
            messages=[{"role": "user", "content": prompt}]
        )
        
        self.context.ai_assessment = response.choices[0].message.content
        self.context.fraud_score = 0.3 if "HIGH" in self.context.ai_assessment else 0.1
        
        print(f"   Assessment: {self.context.ai_assessment}")
        print(f"   Fraud Score: {self.context.fraud_score:.1f}")
        
        self.context.transition(ClaimState.VERIFICATION)
    
    def verify(self):
        """Step 3: Verify against policy rules"""
        print(f"\n3️⃣ VERIFICATION: Checking policy rules...")
        
        # Use data from Step 1 and Step 2
        is_new_customer = "1 month" in self.context.description or "new" in self.context.description.lower()
        high_amount = self.context.amount > 30000
        high_fraud_risk = self.context.fraud_score > 0.2
        
        print(f"   • New customer: {is_new_customer}")
        print(f"   • High amount: {high_amount}")
        print(f"   • High fraud risk: {high_fraud_risk}")
        
        self.context.transition(ClaimState.DECISION)
    
    def decide(self):
        """Step 4: Make final decision using all context"""
        print(f"\n4️⃣ DECISION: Making approval decision...")
        
        is_new_customer = "1 month" in self.context.description or "new" in self.context.description.lower()
        high_amount = self.context.amount > 30000
        high_fraud_risk = self.context.fraud_score > 0.2
        
        if high_fraud_risk and high_amount:
            self.context.final_decision = "⏳ ESCALATE TO INVESTIGATOR"
        elif is_new_customer and high_amount:
            self.context.final_decision = "⏳ REQUEST MORE DOCUMENTATION"
        elif high_fraud_risk:
            self.context.final_decision = "❌ DENY"
        else:
            self.context.final_decision = "✅ APPROVE"
        
        print(f"   Decision: {self.context.final_decision}")
        self.context.transition(ClaimState.COMPLETED)
    
    def process(self):
        """Run complete workflow"""
        print(f"\n{'='*55}")
        print(f"🛡️  INSURANCE CLAIM: {self.context.customer}")
        print(f"{'='*55}")
        
        if self.intake():
            self.assess()
            self.verify()
            self.decide()
        else:
            print("❌ Claim rejected at intake stage")
        
        print(f"\n📋 Workflow State History:")
        for step in self.context.state_history:
            print(f"   • {step}")
        
        return self.context

print("✅ ClaimProcessor defined!")

✅ ClaimProcessor defined!


## Run the Demo

In [5]:
print("🛡️  INSURANCE CLAIM PROCESSING DEMO")
print("="*55)

for claim in CLAIMS:
    processor = ClaimProcessor(claim)
    processor.process()

print(f"\n✅ All claims processed!")

🛡️  INSURANCE CLAIM PROCESSING DEMO

🛡️  INSURANCE CLAIM: John Smith

1️⃣ INTAKE: Validating claim CLM-001...
   ✅ Valid claim intake
      • Claim: Auto Accident
      • Amount: $5,000

2️⃣ ASSESSMENT: AI analyzing claim...
   Assessment: Risk Assessment: LOW  
Key Red Flags: None identified; the claimant has a clean driving record and is a two-year customer, which supports the legitimacy of the claim.
   Fraud Score: 0.1

3️⃣ VERIFICATION: Checking policy rules...
   • New customer: False
   • High amount: False
   • High fraud risk: False

4️⃣ DECISION: Making approval decision...
   Decision: ✅ APPROVE

📋 Workflow State History:
   • intake → assessment
   • assessment → verification
   • verification → decision
   • decision → completed

🛡️  INSURANCE CLAIM: Jane Doe

1️⃣ INTAKE: Validating claim CLM-002...
   ✅ Valid claim intake
      • Claim: Water Damage
      • Amount: $35,000

2️⃣ ASSESSMENT: AI analyzing claim...
   Assessment: Risk Assessment: LOW  
Key Red Flags: None ide

## Comparison: Without State Management

What happens if we try to process claims **without** state management?

In [6]:
print("\n" + "="*60)
print("❌ PROBLEM: Without State Management")
print("="*60)

def broken_claim_processing(claim):
    """BAD approach: No state management"""
    print(f"\n🔴 Processing {claim['customer']} without state management...")
    
    # Step 1: Intake
    print(f"  Step 1: Validate claim")
    valid = claim["policy_active"] and claim["amount"] > 0
    print(f"  ✓ Validated: {valid}")
    
    # Step 2: Assess - but we lost validation results!
    print(f"  Step 2: AI assessment")
    print(f"  ⚠️  Had to recreate claim data for the prompt")
    print(f"  ⚠️  Can't verify if Step 1 passed")
    
    # Step 3: Verify - but we lost assessment!
    print(f"  Step 3: Policy verification")
    print(f"  ⚠️  Lost the AI assessment from Step 2")
    print(f"  ⚠️  Lost validation results from Step 1")
    
    # Step 4: Decide - complete data loss!
    print(f"  Step 4: Make decision")
    print(f"  ⚠️  Had to recalculate fraud score")
    print(f"  ⚠️  No audit trail of what happened")
    print(f"  ⚠️  Can't resume if workflow fails")
    print(f"  ⚠️  Each step is isolated - no context!")

broken_claim_processing(CLAIMS[0])

print("\n" + "="*60)
print("✅ SOLUTION: With State Management")
print("="*60)
print("✓ Context preserved across all steps")
print("✓ Each step uses verified data from previous steps")
print("✓ Complete audit trail of all decisions")
print("✓ Easy to debug and resume workflows")
print("✓ Data flows naturally through the state machine")


❌ PROBLEM: Without State Management

🔴 Processing John Smith without state management...
  Step 1: Validate claim
  ✓ Validated: True
  Step 2: AI assessment
  ⚠️  Had to recreate claim data for the prompt
  ⚠️  Can't verify if Step 1 passed
  Step 3: Policy verification
  ⚠️  Lost the AI assessment from Step 2
  ⚠️  Lost validation results from Step 1
  Step 4: Make decision
  ⚠️  Had to recalculate fraud score
  ⚠️  No audit trail of what happened
  ⚠️  Can't resume if workflow fails
  ⚠️  Each step is isolated - no context!

✅ SOLUTION: With State Management
✓ Context preserved across all steps
✓ Each step uses verified data from previous steps
✓ Complete audit trail of all decisions
✓ Easy to debug and resume workflows
✓ Data flows naturally through the state machine


## Key Insights

### State Management: A Universal Pattern

This insurance claim processing system demonstrates the core state management pattern:

**4-Step Workflow**:
- INTAKE → ASSESSMENT → VERIFICATION → DECISION → COMPLETED

**Key Components**:
1. ✅ Define states (Enum)
2. ✅ Create context dataclass (preserves data)
3. ✅ Implement state handlers (workflow steps)
4. ✅ Maintain state history (audit trail)
5. ✅ Integrate LLM at specific points (AI-assisted decisions)

### Why This Pattern Matters

Any workflow with:
- Multiple sequential steps
- Data flowing between steps
- Decisions based on accumulated information
- Need for audit trails
- LLM integration for intelligent decisions

**Should use state management**.

### Real-World Applications

This pattern applies to many financial services workflows:
- 💰 Loan and credit approvals
- 🛡️ Insurance claim processing (this demo)
- 📋 KYC/AML compliance reviews
- 📄 Document verification workflows
- 🔍 Fraud investigation pipelines
- 📞 Customer service escalations

### Production Considerations

Real production systems would add:
- **Persistence**: Database storage for state
- **Resilience**: Error recovery and retries
- **Scalability**: Queue-based async processing
- **Observability**: Monitoring, metrics, and alerts
- **Compliance**: Regulatory logging and audit trails
- **Human-in-Loop**: Escalation and handoff workflows

But the **core pattern remains the same**!